In [1]:
#import necessary modules
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import glob
import pathlib


In [2]:
#Data set parameters
batch_size = 27
img_height = 400
img_width = 506

#paths
train_p=r'C:/Users/nikhi/OneDrive/Desktop/Sem4/data/classification/train1/'
val_p=r'C:/Users/nikhi/OneDrive/Desktop/Sem4/data/classification/val1/'

In [3]:
#training data directory
train_dir=pathlib.Path(train_p).with_suffix('')

#validation data directory
val_dir=pathlib.Path(val_p).with_suffix('')
#len(list(train_dir.glob('*/*.jpg')))

In [4]:
#generate training data --> contains Images and labels
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
num_classes=len(train_ds.class_names)

Found 45095 files belonging to 102 classes.


In [5]:
#generate validation dataset from the directory
val_ds = tf.keras.utils.image_dataset_from_directory(
  val_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 7508 files belonging to 102 classes.


In [6]:
#Normalization of traning data
normalization_layer = tf.keras.layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [7]:
#To use the memory efficiently
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
# Builing the model
model=tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(img_height,img_width,3),
    pooling=max,
    classes=num_classes,
    classifier_activation='softmax',
)

In [9]:
#compiling the model with optimization
from tensorflow.keras.optimizers import Adam
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [10]:
model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400, 506, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 400, 506, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 400, 506, 3)  7           ['rescaling_1[0][0]']            
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 401, 507, 3)  0           ['normalization[0][0

                                                                                                  
 block2a_project_bn (BatchNorma  (None, 100, 127, 24  96         ['block2a_project_conv[0][0]']   
 lization)                      )                                                                 
                                                                                                  
 block2b_expand_conv (Conv2D)   (None, 100, 127, 14  3456        ['block2a_project_bn[0][0]']     
                                4)                                                                
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 100, 127, 14  576        ['block2b_expand_conv[0][0]']    
 ization)                       4)                                                                
                                                                                                  
 block2b_e

                                                                                                  
 block3b_expand_conv (Conv2D)   (None, 50, 64, 240)  9600        ['block3a_project_bn[0][0]']     
                                                                                                  
 block3b_expand_bn (BatchNormal  (None, 50, 64, 240)  960        ['block3b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block3b_expand_activation (Act  (None, 50, 64, 240)  0          ['block3b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block3b_dwconv (DepthwiseConv2  (None, 50, 64, 240)  6000       ['block3b_expand_activation[0][0]
 D)       

 ivation)                                                                                         
                                                                                                  
 block4b_dwconv (DepthwiseConv2  (None, 25, 32, 480)  4320       ['block4b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block4b_bn (BatchNormalization  (None, 25, 32, 480)  1920       ['block4b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block4b_activation (Activation  (None, 25, 32, 480)  0          ['block4b_bn[0][0]']             
 )                                                                                                
          

 block5a_bn (BatchNormalization  (None, 25, 32, 480)  1920       ['block5a_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block5a_activation (Activation  (None, 25, 32, 480)  0          ['block5a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block5a_se_squeeze (GlobalAver  (None, 480)         0           ['block5a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
          

                                                                                                  
 block5c_se_reduce (Conv2D)     (None, 1, 1, 28)     18844       ['block5c_se_reshape[0][0]']     
                                                                                                  
 block5c_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block5c_se_reduce[0][0]']      
                                                                                                  
 block5c_se_excite (Multiply)   (None, 25, 32, 672)  0           ['block5c_activation[0][0]',     
                                                                  'block5c_se_expand[0][0]']      
                                                                                                  
 block5c_project_conv (Conv2D)  (None, 25, 32, 112)  75264       ['block5c_se_excite[0][0]']      
                                                                                                  
 block5c_p

                                )                                 'block6b_se_expand[0][0]']      
                                                                                                  
 block6b_project_conv (Conv2D)  (None, 13, 16, 192)  221184      ['block6b_se_excite[0][0]']      
                                                                                                  
 block6b_project_bn (BatchNorma  (None, 13, 16, 192)  768        ['block6b_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6b_drop (Dropout)         (None, 13, 16, 192)  0           ['block6b_project_bn[0][0]']     
                                                                                                  
 block6b_add (Add)              (None, 13, 16, 192)  0           ['block6b_drop[0][0]',           
          

                                                                                                  
 block6d_project_bn (BatchNorma  (None, 13, 16, 192)  768        ['block6d_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6d_drop (Dropout)         (None, 13, 16, 192)  0           ['block6d_project_bn[0][0]']     
                                                                                                  
 block6d_add (Add)              (None, 13, 16, 192)  0           ['block6d_drop[0][0]',           
                                                                  'block6c_add[0][0]']            
                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 13, 16, 1152  221184      ['block6d_add[0][0]']            
          

In [11]:
# run to resume training from previously saved weights
from keras.models import load_model
model=load_model("model_rec1.h5")

In [ ]:
#Training 
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

In [ ]:
# Run to save the model weights 
model.save("model_rec1.h5")
model.save_weights("model_weightsrec1.h5")

In [12]:
#Testing 
test_data = r'C:/Users/nikhi/OneDrive/Desktop/Sem4/data/classification/test1/'
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_data,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
num_classes=len(test_ds.class_names)


Found 22618 files belonging to 102 classes.


In [ ]:
#Accuracy metrics 
test_loss, test_accuracy = model.evaluate(test_ds)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

C:\Users\nikhi\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


 29/838 [>.............................] - ETA: 1:18:14 - loss: 4.6424 - accuracy: 0.0754